In [1]:
import pandas as pd
import numpy as np
from uszipcode import SearchEngine
from collections import Counter
import mpld3 
mpld3.enable_notebook()
import os
import math
from sklearn.preprocessing import Imputer
%config IPCompleter.greedy=True
import xgboost as xgb
os.environ['KMP_DUPLICATE_LIB_OK']='True'

os.chdir("/Users/mirbek/Desktop/Data Science/ubiqum/projects/DS5/Task1")
print(os.listdir("/Users/mirbek/Desktop/Data Science/ubiqum/projects/DS5/Task1"))

['.DS_Store', 'SBAnational.csv', 'Should This Loan be Approved or Denied A Large Dataset with Class Assignment Guidelines.pdf', 'credit_clean.csv']


In [18]:
credit = pd.read_csv('credit_clean.csv', header ='infer')

In [40]:
credit.head()

,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,LowDoc,MIS_Status,GrAppv,Sector_Points,State_Points,IscreateJob,IsRetained,IsFranchise
0,7,4,2.0,0,0,0,1,1,60000.0,2,3,0,0,1
1,5,2,2.0,0,0,0,1,1,40000.0,2,3,0,0,1
2,15,7,1.0,0,0,0,0,1,287000.0,5,3,0,0,1
3,5,2,1.0,0,0,0,1,1,35000.0,0,3,0,0,1
4,20,14,1.0,7,7,0,0,1,229000.0,0,1,1,1,1


In [19]:
#Fix skewness of buy_price, sq_mt_built, Avg_price_neigh and Total_Mean
credit['GrAppv'] = np.log(credit['GrAppv']) 
#credit = credit.drop(axis =1, columns = ['NoEmp', 'GrAppv'])

In [20]:
#Split data into train and test set + label target value
from sklearn.model_selection import train_test_split
y = credit.MIS_Status
X = credit.drop(['MIS_Status'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=0)

In [21]:
my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

/Users/mirbek/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
#Scaling features with Standart Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_sc =scaler.fit_transform(train_X)
test_sc = scaler.transform(test_X)

In [27]:
#Train the XGboost Model for Classification
#Model with default parameters
model1 = xgb.XGBClassifier()
#Model with with tuned parameters
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(train_sc, train_y)
train_model2 = model2.fit(train_sc, train_y)

In [28]:
#prediction and Classification Report
from sklearn.metrics import classification_report

pred1 = train_model1.predict(test_X)
pred2 = train_model2.predict(test_X)

print('Model 1 XGboost Report %r' % (classification_report(test_y, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(test_y, pred2)))

/Users/mirbek/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mirbek/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mirbek/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model 1 XGboost Report '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00     39352\n           1       0.82      1.00      0.90    184649\n\n   micro avg       0.82      0.82      0.82    224001\n   macro avg       0.41      0.50      0.45    224001\nweighted avg       0.68      0.82      0.74    224001\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n           0       0.25      0.00      0.00     39352\n           1       0.82      1.00      0.90    184649\n\n   micro avg       0.82      0.82      0.82    224001\n   macro avg       0.54      0.50      0.45    224001\nweighted avg       0.72      0.82      0.75    224001\n'


In [29]:
#Let's use accuracy score
from sklearn.metrics import accuracy_score

print("Accuracy for model 1: %.2f" % (accuracy_score(test_y, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(test_y, pred2) * 100))

Accuracy for model 1: 82.43
Accuracy for model 2: 82.38
